<a href="https://colab.research.google.com/github/geoskimoto/AWDB_SOAP_Request/blob/main/NSteele_EstimatingNulls_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run all cells under the Imports and Functions headers first: 

*   Click the arrow to run all cells under header
*   Or hit shift + enter for each and every individual cell.

# Imports

In [82]:
try:
    import xmltodict
    print("module 'xmltodict' is installed")
except ModuleNotFoundError:
  !pip install xmltodict

module 'xmltodict' is installed


In [83]:
from sklearn.linear_model import LassoCV, RidgeCV, HuberRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import scipy.stats as stats
# import statsmodels.api as sm
from functools import reduce

import requests
import xmltodict
import datetime

# Functions

## Helper Functions

In [84]:
# Web Call to Access and Download Data of a Single Station from AWDB Web Service (SOAP API)

def SOAP_Call(stationtriplets, elementCD, begindate, enddate):
  global xml, dict_of_xml, df
  # Create a dictionaries to store the data
  headers = {'Content-type': 'text/soap'}
  # current_dictionary = {}
  
  # Define Web Service URL
  URL = "https://wcc.sc.egov.usda.gov/awdbWebService/services?WSDL"

  # Define Parameters for SOAP Elements (getData:current and getCentralTendencyData:normals)
  SOAP_current = '''
  <?xml version="1.0" encoding="UTF-8"?>
  <SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:q0="http://www.wcc.nrcs.usda.gov/ns/awdbWebService" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
    <SOAP-ENV:Body>
      <q0:getData>
        <stationTriplets>STATIONTRIPLETS</stationTriplets>
        <elementCd>ELEMENTCD</elementCd>   
        <ordinal>1</ordinal>
        <duration>DAILY</duration>
        <getFlags>false</getFlags>
        <beginDate>BEGINDATE</beginDate>
        <endDate>ENDDATE</endDate>
        <alwaysReturnDailyFeb29>false</alwaysReturnDailyFeb29>   
      </q0:getData>
    </SOAP-ENV:Body>
  </SOAP-ENV:Envelope>

  '''.strip()
  #Read GetData documents - If <alwaysReturnDailyFeb29> is set to true, will set a null for every non leap year on the 29th,  
  #which breaks this request when selecting date ranges that include Feb 29.
  #Possible element codes: PREC, WTEQ (Water Equivalent/SWE)
  
  # Post SOAP Elements to AWDB Web Service and process results - getData
  SOAP_current = SOAP_current.replace("ELEMENTCD", elementCD)
  SOAP_current = SOAP_current.replace("STATIONTRIPLETS", stationtriplets)
  SOAP_current = SOAP_current.replace("BEGINDATE", begindate)
  SOAP_current = SOAP_current.replace("ENDDATE", enddate)

  #Send request to server and receive xml document
  xml = requests.post(URL, data=SOAP_current, headers=headers)

  #convert xml document to a dictionary, extract values putting them in a dataframe.  XML's aren't the easiest to parse and extract data from, so this is a nice work around.
  dict_of_xml = xmltodict.parse(xml.text)
  df = dict_of_xml['soap:Envelope']['soap:Body']['ns2:getDataResponse']['return']['values']

  #Null values are given as OrderedDictionaries with lots of text, while actual values are given as strings.  This converts all the OrderedDictionaries into actual null/none values, and converts all values that were given as strings into float numbers.
  df = pd.DataFrame(map(lambda i: float(i) if type(i) == str else None, df))

  #Since invidual dates aren't associated with the values in the xml document, have to create a range of dates bw the begindate and endate, which is then added to the dataframe.
  df['Date'] = pd.date_range(begindate,enddate,freq='d')
  df.columns = [f'{elementCD}','Date']
  df.set_index('Date', inplace=True)

  return df


In [85]:
# Function to Download Multiple Stations at a time from AWDB Web Service

def getData(stations, parameter_of_interest, begindate, enddate):
  data_singleDF = reduce(lambda left,right: pd.merge(left,right,left_index=True, right_index=True, how='outer'), [SOAP_Call(stationtriplets=j,elementCD=parameter_of_interest,begindate=begindate,enddate=enddate) for j in stations])
  data_singleDF.columns = [f'{j}' for j in stations]
  return data_singleDF

In [86]:
#Choose Regression Model

def regressionModel(regression_model):
  if regression_model == 'Lasso':
    regr = LassoCV(alphas=(0.001,0.01,0.1,1,10,100,1000))
    return regr
  elif regression_model == 'Ridge':
    regr = RidgeCV(alphas=(0.001,0.01,0.1,1,10,100,1000))
    return regr
  elif regression_model == 'SVM':  #Support vector machines
    regr = svm.SVR(kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)
    return regr
  elif regression_model == 'Huber':
    regr = HuberRegressor()
    return regr
  elif regression_model == 'GradientBoost':
    regr = GradientBoostingRegressor(random_state=0, n_estimators=100)
    return regr
  elif regression_model == 'AdaBoost':
    regr = AdaBoostRegressor(random_state=0, n_estimators=100)
    return regr
  elif regression_model == 'Random Forest':
    regr = RandomForestRegressor(n_estimators=100)
    return regr
  else:
    print('Choose either Lasso, Ridge, SVM, Huber, GradientBoost, AdaBoost, or Random Forest')

## Regression Functions

### Function to check model fit

In [98]:
def checkModel(stations, parameter_of_interest, begindate, enddate, regression_model, test_size):

#Download Data from AWDB Web Service
  data = getData(stations, parameter_of_interest, begindate, enddate).dropna()

#Define Targets and Features (e.g. Dependent and Independent Variables)
  target = data.iloc[:,0]
  features = data.iloc[:,1:]

#Split into training and test sets
  features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=test_size, shuffle=False)
  # x_train, x_test, y_train, y_test
# Choose Regression Model:

  regr = regressionModel(regression_model)

#Fit model on training set
  regr.fit(features_train, target_train)

#Run predictions on training features and test features
  target_train_pred = regr.predict(features_train)
  target_test_pred = regr.predict(features_test)

#Print Root Mean Square Error for training and test sets:
  print('RMSE for training set', mean_squared_error(target_train, target_train_pred))
  print('RMSE for test set', mean_squared_error(target_test, target_test_pred))

#Plot Data

  fig = go.Figure()
  fig.add_trace(go.Scatter(
      y = target_train,
      x = target_train.index,
      mode = 'lines',
      name = 'Training Data'
  ))

  fig.add_trace(go.Scatter(
      y = target_train_pred,
      x = target_train.index,
      mode = 'lines',
      name = 'Model Predictions on Training Data'
  ))

  fig.add_trace(go.Scatter(
      y = target_test,
      x = target_test.index,
      mode = 'lines',
      name = 'Test Data'
  ))

  fig.add_trace(go.Scatter(
      y = target_test_pred,
      x = target_test.index,
      mode = 'lines',
      name = 'Model Predictions on Test Data'
  ))

  fig.update_yaxes(title_text = f"{parameter_of_interest}")
  fig.update_xaxes(title_text = 'Date') 

  fig.update_layout(
      showlegend=True,
      height=450,
      width=1200 
  )
  fig.show()

  # # # # # Regression Plot # # # # #

  fig2 = go.Figure()
  fig2.add_trace(go.Scatter(
    x=features_train.iloc[:,0],
    y=target_train,
    mode='markers',
    hovertext = features_train.index,
    name =  'Response vs. Predictors'
  ))

  fig2.add_trace(go.Scatter(
    x=features_train.iloc[:,0],
    y=pd.DataFrame(target_train_pred.tolist()).iloc[:,0],
    mode='lines',
    hovertext = features_train.index,
    name = 'Model Fit'  
  ))
  
  fig2.update_yaxes(title_text = f'{stations[1:]} {parameter_of_interest}')
  fig2.update_xaxes(title_text = f'{stations[0]} {parameter_of_interest}')
  fig2.update_layout(
      showlegend=True,
      height=450,
      width=1200 
  )
  fig2.show()

  return regr
  

In [99]:
 #Thunder Basin Predictions

stations = ['817:WA:SNTL', '711:WA:SNTL']#, '975:WA:SNTL']  #975 pulls SWE down while 681 greatly pushes it up.  711 slightly raises it.  '681:WA:SNTL']#,
parameter_of_interest = 'WTEQ'
begindate = '09/01/2020'
enddate = '02/28/2021'

regr = checkModel(stations, parameter_of_interest, begindate, enddate, regression_model='Ridge', test_size=0.2)

RMSE for training set 0.46747610627962666
RMSE for test set 0.3892933307652291


In [ ]:
regr2.

In [89]:
 #Thunder Basin Predictions

stations = ['817:WA:SNTL', '681:WA:SNTL']#, '975:WA:SNTL']  #975 pulls SWE down while 681 greatly pushes it up.  711 slightly raises it.  '681:WA:SNTL']#,
parameter_of_interest = 'WTEQ'
begindate = '09/01/2020'
enddate = '02/28/2021'

checkModel(stations, parameter_of_interest, begindate, enddate, regression_model='Ridge', test_size=0.2)

RMSE for training set 0.7284804441768696
RMSE for test set 4.78093050532517


In [90]:
 #Thunder Basin Predictions

stations = ['817:WA:SNTL','975:WA:SNTL']  #975 pulls SWE down while 681 greatly pushes it up.  711 slightly raises it.  '681:WA:SNTL']#,
parameter_of_interest = 'WTEQ'
begindate = '09/01/2020'
enddate = '02/28/2021'

checkModel(stations, parameter_of_interest, begindate, enddate, regression_model='Ridge', test_size=0.2)

RMSE for training set 0.5685531663074581
RMSE for test set 3.8012742723229533


In [91]:
 #Thunder Basin Predictions

stations = ['817:WA:SNTL', '711:WA:SNTL', '681:WA:SNTL','975:WA:SNTL']  #975 pulls SWE down while 681 greatly pushes it up.  711 slightly raises it.  '681:WA:SNTL']#,
parameter_of_interest = 'WTEQ'
begindate = '09/01/2020'
enddate = '02/28/2021'

checkModel(stations, parameter_of_interest, begindate, enddate, regression_model='Ridge', test_size=0.2)

RMSE for training set 0.1037606547019246
RMSE for test set 0.7314687508428687


In [106]:
def makePredictions(predict_begindate, predict_enddate, regression_model = regr, stations = stations):

  # global predict_target, predict_features, train_target, train_features

#Download Data from AWDB Web Service
  predict_data  = getData(stations, parameter_of_interest, predict_begindate, predict_enddate)

  predict_target = predict_data.iloc[:,0].fillna(0)
  predict_features = predict_data.iloc[:,1:].fillna(0)

#Run predictions
  predictions = regr.predict(predict_features)
  
#Plot predictions 
  fig = go.Figure()

  fig.add_trace(go.Scatter(
      y = predict_target,
      x = predict_target.index,
      mode = 'lines',
      name = f'{stations[0]} {parameter_of_interest}'
  )) 

  fig.add_trace(go.Scatter(
      y = predictions,
      x = predict_features.index,
      mode = 'lines',
      name = 'Model Predictions'
  )) 

  fig.update_layout(
    showlegend=True,
    height=650,
    width=1400,
    title={
        'text': "Model Predictions",
        'xanchor': 'center',
        'yanchor': 'top',
        'y':0.9,
        'x':0.4},
    xaxis_title = "Date",
    yaxis_title = f"{parameter_of_interest} (in)"
  )

  fig.show()

In [107]:
makePredictions('01/01/2021','5/01/2021')

### Function to fit model on selected data, and then make predictions on separate data.

In [100]:
def regr_Predictions(stations, parameter_of_interest, train_begindate, train_enddate, predict_begindate, predict_enddate, regression_model):

  # global predict_target, predict_features, train_target, train_features

#Download Data from AWDB Web Service
  train_data = getData(stations, parameter_of_interest, train_begindate, train_enddate)
  predict_data  = getData(stations, parameter_of_interest, predict_begindate, predict_enddate)
#Define Targets and Features (e.g. Dependent and Independent Variables)
  train_target = train_data.iloc[:,0].fillna(0)
  train_features = train_data.iloc[:,1:].fillna(0)

  predict_target = predict_data.iloc[:,0].fillna(0)
  predict_features = predict_data.iloc[:,1:].fillna(0)

# Choose Regression Model:
  # regr = regr
  # regr = regressionModel(regression_model)

#Fit model to training set
  regr.fit(train_features, train_target)

#Run predictions
  predictions = regr.predict(predict_features)
  
#Plot predictions 
  fig = go.Figure()

  fig.add_trace(go.Scatter(
      y = predict_target,
      x = predict_target.index,
      mode = 'lines',
      name = f'{stations[0]} {parameter_of_interest}'
  )) 

  fig.add_trace(go.Scatter(
      y = predictions,
      x = predict_features.index,
      mode = 'lines',
      name = 'Model Predictions'
  )) 

  fig.update_layout(
    showlegend=True,
    height=650,
    width=1400,
    title={
        'text': "Model Predictions",
        'xanchor': 'center',
        'yanchor': 'top',
        'y':0.9,
        'x':0.4},
    xaxis_title = "Date",
    yaxis_title = f"{parameter_of_interest} (in)"
  )

  fig.show()

# Estimating Missing Data

In [ ]:
#Blewett Pass Predictions

#Make first station your dependent variable (e.g. the station with missing data that you are trying estimate).  
stations = ['352:WA:SNTL','507:WA:SNTL']#, '841:WA:SNTL','478:WA:SNTL']  #Don't use Sassy (841), as it will overestimate WTEQ
parameter_of_interest = 'WTEQ'  #Can choose WTEQ, SNWD, PREC, PRCP, TAVG, TMIN, TMAX, TOBS...and many others (https://www.nrcs.usda.gov/wps/portal/wcc/home/dataAccessHelp/webService/webServiceReference/!ut/p/z1/jc9NDoIwEAXgs3CCPqCWuqxNBFIiqdCAsyFdkSaKLozn1xg2LmyY3STfmx9GbGS0-FeY_TPcF3_99BcSk5Sapy1HU576HFactTtWJkMKNnxBnUlVyQ6m1JxD7XXRN10BcwCjLXn8KbUxHwEUHz8w-l0BWWew2hre7pBDpyuIvRgFvVhB5IrHzbkRoZ5VkrwBnC-XyQ!!/?1dmy&current=true&urile=wcm%3apath%3a%2Fwcc%2Bcontent%2Fhome%2Fdata%2Baccess%2Bhelp%2Fweb%2Bservice%2Fweb%2Bservice%2Breference#elementCodes).

# **NOTE** The following four dates can not be NULL in database (if you see a value error, this is why).  
# Usually the predict_enddate will have a NULL value (as this date is usually within the date range that you are trying to estimate).  To get function to run, go into DMP and change the value to an arbritary number and save.  After running predictions, make sure to change the arbritary value in DMP to an actual estimate.
#Define the range of dates to train the model.
train_begindate = '10/01/2020'    
train_enddate = '02/26/2021'
#Define the range of dates for which you'd like to make predictions.
predict_begindate = '01/01/2021'  
predict_enddate = '05/01/2021'

regr_Predictions(stations, parameter_of_interest, train_begindate, train_enddate, predict_begindate, predict_enddate, regression_model='Random Forest') 
# regression_model options include:
  # Linear models: Lasso, Ridge, Huber. 
  # Ensemble models: GradientBoost, AdaBoost, or Random Forest
  # Non linear models: SVM 

In [ ]:
## Aneroid Lake 

stations = ['302:OR:SNTL', '653:OR:SNTL']
parameter_of_interest = 'WTEQ'
train_begindate = '12/01/2019'
train_enddate = '11/29/2020'
predict_begindate = '11/01/2020'
predict_enddate = '05/01/2021'

regr_Predictions(stations, parameter_of_interest, train_begindate, train_enddate, predict_begindate, predict_enddate, regression_model='Ridge')

In [ ]:
#Morse Lake Predictions

stations = ['642:WA:SNTL', '1085:WA:SNTL']#,'679:WA:SNTL']
parameter_of_interest = 'WTEQ'
train_begindate = '12/01/2019'
train_enddate = '12/22/2020'
predict_begindate = '12/22/2020'
predict_enddate = '05/01/2021'

regr_Predictions(stations, parameter_of_interest, train_begindate, train_enddate, predict_begindate, predict_enddate, regression_model='Ridge')

In [ ]:
 #Mt. Crag Predictions

stations = ['648:WA:SNTL', '1107:WA:SNTL','974:WA:SNTL']
parameter_of_interest = 'WTEQ'
train_begindate = '11/10/2019'
train_enddate = '12/10/2020'
predict_begindate = '10/02/2020'
predict_enddate = '04/29/2021'


regr_Predictions(stations, parameter_of_interest, train_begindate, train_enddate, predict_begindate, predict_enddate, regression_model='Ridge')

In [ ]:
 #Olallie Meadows Predictions

stations = ['672:WA:SNTL', '788:WA:SNTL', '898:WA:SNTL']
parameter_of_interest = 'WTEQ'
train_begindate = '01/01/2021'
train_enddate = '03/31/2021'
predict_begindate = '01/01/2021'
predict_enddate = '05/01/2021'


regr_Predictions(stations, parameter_of_interest, train_begindate, train_enddate, predict_begindate, predict_enddate, regression_model='Ridge')

In [101]:
 #Thunder Basin Predictions

stations = ['817:WA:SNTL', '711:WA:SNTL', '975:WA:SNTL']  #975 pulls SWE down while 681 greatly pushes it up.  711 slightly raises it.  '681:WA:SNTL']#,
parameter_of_interest = 'PREC'
train_begindate = '01/01/2021'
train_enddate = '02/28/2021'
predict_begindate = '01/01/2021'
predict_enddate = '5/01/2021'

regr_Predictions(stations, parameter_of_interest, train_begindate, train_enddate, predict_begindate, predict_enddate, regression_model='Ridge')